## VGG-16

In [3]:
import numpy as np
import pandas as pd

import torch
from torch import optim, cuda
from torch.utils.data import DataLoader, sampler, random_split

import torch.nn as nn
import torchvision
from torchvision import transforms, models



In [4]:
if torch.backends.mps.is_available():
    print("MPS (Metal Performance Shaders) GPU is available.")
else:
    print("No GPU is available.")

MPS (Metal Performance Shaders) GPU is available.


In [21]:
# define transform for train val and test
image_transforms = {
    # training with augmentation
    'train':
        transforms.Compose([
            transforms.Resize(224),
            transforms.Grayscale(3),
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
            transforms.Normalize((0.5,), (0.5,))
        ]),
    'val':
        transforms.Compose([
            transforms.Resize(224),
            transforms.Grayscale(3),
            transforms.ToTensor(),
            transforms.Normalize((0.5,), (0.5,))
        ]),
    'test':
        transforms.Compose([
            transforms.Resize(224),
            transforms.Grayscale(3),
            transforms.ToTensor(),
            transforms.Normalize((0.5,), (0.5,))
        ])
}

fashion_full_train = torchvision.datasets.FashionMNIST('../data', train=True, download=True)
fashion_test = torchvision.datasets.FashionMNIST('../data', train=False, transform=image_transforms['test'], download=True)


In [22]:
print(len(fashion_test))

10000


In [23]:
class_mapping = {
    0: 'T-shirt',
    1: 'Trouser',
    2: 'Pullover',
    3: 'Dress',
    4: 'Coat',
    5: 'Sandal',
    6: 'Shirt',
    7: 'Sneaker',
    8: 'Bag',
    9: 'Ankle Boot'
}

In [24]:

fashion_train, fashion_val = random_split(fashion_full_train, [int(len(fashion_full_train)*0.8), int(len(fashion_full_train)*0.2)])

fashion_train.dataset.transform = image_transforms['train']
fashion_val.dataset.transform = image_transforms['val']


In [25]:
batch_size = 8

train_loader = DataLoader(fashion_train, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(fashion_val, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(fashion_test, batch_size=batch_size, shuffle=True)

In [39]:
# Get a single batch from the train_loader
data_iter = iter(train_loader)
images, labels = next(data_iter)

# Check the structure of the images and labels
print(f"Images shape: {images.shape}")
print(f"Labels shape: {labels.shape}")

Images shape: torch.Size([8, 3, 224, 224])
Labels shape: torch.Size([8])


In [36]:
model_vgg16 = models.vgg16(weights='DEFAULT')

Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /Users/ygao/.cache/torch/hub/checkpoints/vgg16-397923af.pth
8.1%


KeyboardInterrupt: 

In [31]:
model_vgg16

NameError: name 'model_vgg16' is not defined

In [ ]:
# Freeze early layers
for param in model_vgg16.parameters():
    param.requires_grad = False

In [ ]:
n_classes = 10
n_inputs = model_vgg16.classifier[6].in_features
model_vgg16.classifier[6] = nn.Sequential(
    nn.Linear(n_inputs, 256),
    nn.ReLU(),
    nn.Dropout(0.4),
    nn.Linear(256, n_classes),
    nn.LogSoftmax(dim=1))

model_vgg16.classifier

In [ ]:
# summary(model_vgg16, input_size=(3, 224, 224), batch_size=batch_size, device='cuda')

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model_vgg16.parameters(), lr=0.001)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)
device = (
    torch.device("mps") if torch.backends.mps.is_available() else
    torch.device("cuda") if torch.cuda.is_available() else
    torch.device("cpu")
)
model_vgg16 = model_vgg16.to(device)
# model_vgg16.cuda()

In [ ]:
num_epochs = 5
for epoch in range(num_epochs):
    # model_vgg16.train()  # do I want to set the model to training? no
    running_loss = 0.0

    # training loop
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model_vgg16(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    # step the scheduler after each epoch
    scheduler.step()

    # validation loop
    # model_vgg16.eval()  # guess no need for setting
    validation_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():  # gradient computation for validation
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model_vgg16(images)
            loss = criterion(outputs, labels)
            validation_loss += loss.item()

            # calculate accuracy
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    # average losses and accuracy
    avg_training_loss = running_loss / len(train_loader)
    avg_validation_loss = validation_loss / len(val_loader)
    validation_accuracy = 100 * correct / total

    print(f"Epoch [{epoch+1}/{num_epochs}], "
          f"Train Loss: {avg_training_loss:.4f}, "
          f"Validation Loss: {avg_validation_loss:.4f}, "
          f"Validation Accuracy: {validation_accuracy:.2f}%")

print('Fine-tuning complete!')

# Save the fine-tuned model
torch.save(model_vgg16.state_dict(), 'finetuned_vgg16_mnist.pth')
print('Model saved!')